<a href="https://colab.research.google.com/github/Praveen8824/Movie-Recommendation-System./blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy scikit-learn




In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Movie /tmdb_5000_credits.csv'
data = pd.read_csv(file_path)

In [ ]:
data.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
import ast

def get_top_cast(cast_str):
    cast_list = ast.literal_eval(cast_str)
    cast_names = [cast_member['name'] for cast_member in cast_list[:3]]
    return ' '.join(cast_names)

data['top_cast'] = data['cast'].apply(get_top_cast)

data[['movie_id', 'title', 'top_cast']].head()


,movie_id,title,top_cast
0,19995,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver
1,285,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley
2,206647,Spectre,Daniel Craig Christoph Waltz Léa Seydoux
3,49026,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman
4,49529,John Carter,Taylor Kitsch Lynn Collins Samantha Morton


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data['features'] = data['title'] + ' ' + data['top_cast']

tfidf_vectorizer = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf_vectorizer.fit_transform(data['features'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

cosine_sim.shape


(4803, 4803)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim, data=data, top_n=10):

    idx = data[data['title'] == title].index[0]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:top_n+1]

    movie_indices = [i[0] for i in sim_scores]

    return data['title'].iloc[movie_indices]

recommended_movies = get_recommendations('Toy Story', top_n=10)
recommended_movies


343               Toy Story 2
42                Toy Story 3
4010       That Thing You Do!
1159    A League of Their Own
2314                      Big
883       Catch Me If You Can
373           Mission to Mars
691                 Wild Hogs
2483             Philadelphia
635                 Apollo 13
Name: title, dtype: object